In [1]:
import heavylight
import pandas as pd
import numpy as np

# A Vectorised Actuarial Contingency Model

In [2]:
class Life(heavylight.Model):
    time_step = 1/12
    def num_pols_if(self, t):
        if t == 0:
            return self.data["initial_policies"]
        else:
            return self.num_pols_if(t - 1) - self.num_deaths(t - 1)
    
    def num_deaths(self, t):
        return self.num_pols_if(t) * self.q_x_m(t)
    
    def age(self, t):
        """age at time t"""
        if t == 0:
            return self.data["initial_age"] # floating point
        else:
            return self.age(t - 1) + Life.time_step
    
    def age_rounded(self, t):
        return np.round(self.age(t)).astype(int)
    
    def q_x_m(self, t):
        """monthly mortality rate"""
        return self.q_x(t) ** (Life.time_step)
    
    def q_x(self, t):
        """annual mortality rate"""
        return self.basis["mortality"][self.age_rounded(t)]

In [3]:
ages = np.array([32+1/12, 42, 25+7/12])
ages

array([32.08333333, 42.        , 25.58333333])

In [4]:
ages ** 1/12

array([2.67361111, 3.5       , 2.13194444])

In [5]:
mortality = heavylight.Table.read_csv("sample_q_x_table.csv")

In [6]:
mortality[np.array([20, 21])]

array([0.141392  , 0.14691122])

In [7]:
basis = {"mortality": mortality}

In [8]:
num_model_points = 3
data = {
    "initial_policies": np.ones(num_model_points),
    "initial_age" : np.array([32+1/12, 42, 25+7/12]),
    }

In [9]:
model = Life(data=data, basis=basis)

In [10]:
model.num_pols_if(2)

array([0.01372978, 0.0078071 , 0.01795082])

In [11]:
model.num_pols_if(20)
model.df

,age,age_rounded,num_deaths,num_pols_if,q_x,q_x_m
0,"[32.083333333333336, 42.0, 25.583333333333332]","[32, 42, 26]","[0.8828258515777705, 0.9116422250059119, 0.866...","[1.0, 1.0, 1.0]","[0.224130264787305, 0.3295286574691513, 0.1779...","[0.8828258515777705, 0.9116422250059119, 0.866..."
1,"[32.16666666666667, 42.083333333333336, 25.666...","[32, 42, 26]","[0.10344436736375484, 0.08055067859218223, 0.1...","[0.11717414842222951, 0.08835777499408815, 0.1...","[0.224130264787305, 0.3295286574691513, 0.1779...","[0.8828258515777705, 0.9116422250059119, 0.866..."
2,"[32.25000000000001, 42.16666666666667, 25.7499...","[32, 42, 26]","[0.01212100565492424, 0.007117278734669153, 0....","[0.013729781058474666, 0.007807096401905911, 0...","[0.224130264787305, 0.3295286574691513, 0.1779...","[0.8828258515777705, 0.9116422250059119, 0.866..."
3,"[32.33333333333334, 42.25000000000001, 25.8333...","[32, 42, 26]","[0.0014202685156367758, 0.0006288669130081055,...","[0.0016087754035504255, 0.0006898176672367577,...","[0.224130264787305, 0.3295286574691513, 0.1779...","[0.8828258515777705, 0.9116422250059119, 0.866..."
4,"[32.41666666666668, 42.33333333333334, 25.9166...","[32, 42, 26]","[0.00016641875385064311, 5.556528120079702e-05...","[0.00018850688791364968, 6.0950754228652235e-0...","[0.224130264787305, 0.3295286574691513, 0.1779...","[0.8828258515777705, 0.9116422250059119, 0.866..."
5,"[32.500000000000014, 42.41666666666668, 25.999...","[33, 42, 26]","[1.956262436532288e-05, 4.909624613823256e-06,...","[2.208813406300657e-05, 5.385473027855218e-06,...","[0.2329254993329104, 0.3295286574691513, 0.177...","[0.8856621527884767, 0.9116422250059119, 0.866..."
6,"[32.58333333333335, 42.500000000000014, 26.083...","[33, 43, 26]","[2.2367483557387096e-06, 4.3520115381033597e-0...","[2.5255096976836874e-06, 4.758484140319622e-07...","[0.2329254993329104, 0.3424971350676757, 0.177...","[0.8856621527884767, 0.9145793933046585, 0.866..."
7,"[32.666666666666686, 42.58333333333335, 26.166...","[33, 43, 26]","[2.5574499174907855e-07, 3.717514659299149e-08...","[2.8876134194497785e-07, 4.064726022162622e-08...","[0.2329254993329104, 0.3424971350676757, 0.177...","[0.8856621527884767, 0.9145793933046585, 0.866..."
8,"[32.75000000000002, 42.666666666666686, 26.249...","[33, 43, 26]","[2.924133179171842e-08, 3.17552357596159e-09, ...","[3.30163501958993e-08, 3.4721136286347324e-09,...","[0.2329254993329104, 0.3424971350676757, 0.177...","[0.8856621527884767, 0.9145793933046585, 0.866..."
9,"[32.83333333333336, 42.75000000000002, 26.3333...","[33, 43, 26]","[3.3433909266629584e-09, 2.7125515043399913e-1...","[3.77501840418088e-09, 2.965900526731422e-10, ...","[0.2329254993329104, 0.3424971350676757, 0.177...","[0.8856621527884767, 0.9145793933046585, 0.866..."
